In [23]:
#!pip install spacy

In [24]:
import datasets
import huggingface_hub

print("datasets version:", datasets.__version__)
print("huggingface_hub version:", huggingface_hub.__version__)


import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy import sparse
import re
import numpy as np
import math
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier

#from sentence_transformers import SentenceTransformer
from sklearn.svm import SVC
import os
import wordninja
#import spacy
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import glob
import language_tool_python
import torch

datasets version: 2.18.0
huggingface_hub version: 0.20.3


In [25]:
import torch
print(torch.cuda.is_available())  # باید True باشد
print(torch.cuda.get_device_name(0))  # نام GPU شما باید نمایش داده شود


True
NVIDIA GeForce GTX 950M


In [26]:
import torch
print(torch.version.cuda)
import torch
print(torch.__version__)
from transformers import __version__
print(__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

11.7
1.13.1+cu117
4.11.3
True
1
NVIDIA GeForce GTX 950M


In [27]:
nltk.download('punkt')                         
nltk.download('stopwords')                     
nltk.download('wordnet')                       
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [28]:
def load_reddit_dataset(data_path, s_size=None, text_column="selftext", label_column="subreddit"):

    csv_files = glob.glob(os.path.join(data_path, "**", "*.csv"), recursive=True)
    print(f"Found {len(csv_files)} CSV files.")

    all_dataframes = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            all_dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    all_data = pd.concat(all_dataframes, ignore_index=True)
    print("Total rows before cleaning:", len(all_data))

    clear_data = all_data.dropna(subset=[text_column, label_column])
    print("Total rows after dropping NA:", len(clear_data))

    if s_size is not None:
        grouped = clear_data.groupby(label_column)
        eligible_groups = [name for name, group in grouped if len(group) >= s_size]
        filtered_data = clear_data[clear_data[label_column].isin(eligible_groups)]
        df_sampled = filtered_data.groupby(label_column).sample(n=s_size, random_state=42)
        df_sampled = df_sampled.reset_index(drop=True)
        print("Sampled class distribution:")
        print(df_sampled[label_column].value_counts())
    else:
        df_sampled = clear_data
        print("Returned all available data without sampling.")
        print(df_sampled[label_column].value_counts())

    return df_sampled[text_column], df_sampled[label_column]

In [29]:
text_data, label_data = load_reddit_dataset("dataset",s_size=2000)


Found 223 CSV files.
Total rows before cleaning: 1852403
Total rows after dropping NA: 1797816
Sampled class distribution:
subreddit
Anxiety         2000
SuicideWatch    2000
depression      2000
lonely          2000
mentalhealth    2000
Name: count, dtype: int64


In [30]:
print(text_data.iloc[0])


A few weeks ago, I finally caved and called my doctor to get back on my anxiety medicine after my anxiety got so bad that I was having panic attacks whenever I left the house. I cried picking it up. I was so ready to feel normal again. 

Well last night I went to the store and felt an anxiety attack coming. Instead of spiraling out of control like I have been, I was easily able to calm down. I can't share this with anyone else so I thought to post it here. I am so happy that I was able to finish shopping and that I didn't end up with a panic attack. 

I don't know why I waited so long to go back on my medicine.


In [31]:
def clean_data(data):
    html_regex = re.compile("<.*?>")
    num_regex = re.compile("\d+")
    pun_re = re.compile("[^\w\s]")

    cleaned_data = []

    for text in data:
        text = re.sub(html_regex, '', text)

        urls = re.findall(r'https?://\S+|www\.\S+', text)
        for url in urls:
            tokens = re.findall(r'[a-zA-Z]{4,}', url)
            if len(tokens) >= 3:
                text = text.replace(url, ' '.join(tokens))
            else:
                text = text.replace(url, '[URL]')

        text = re.sub(num_regex, '', text)

        text = re.sub(pun_re, '', text)

        cleaned_data.append(text)

    return cleaned_data



text_data01 = clean_data(text_data)

text_data01[0]

'A few weeks ago I finally caved and called my doctor to get back on my anxiety medicine after my anxiety got so bad that I was having panic attacks whenever I left the house I cried picking it up I was so ready to feel normal again \n\nWell last night I went to the store and felt an anxiety attack coming Instead of spiraling out of control like I have been I was easily able to calm down I cant share this with anyone else so I thought to post it here I am so happy that I was able to finish shopping and that I didnt end up with a panic attack \n\nI dont know why I waited so long to go back on my medicine'

In [32]:
# tool = language_tool_python.LanguageTool('en-US')

# def correct_text_languagetool(text):
#     matches = tool.check(text)
#     corrected_text = language_tool_python.utils.correct(text, matches)
#     return corrected_text

# def correct_texts_languagetool(text_list):
#     corrected_texts = []
#     for text in text_list:
#         corrected_text = correct_text_languagetool(text)
#         corrected_texts.append(corrected_text)
#     return corrected_texts

# text_data_corrected = correct_texts_languagetool(text_data01)

# print(text_data_corrected[0])

In [33]:
def splited_data(data):
    temp_data = []
    for corpus in data:
        words = " ".join(corpus.split()).split()
        fixed_words = []

        for word in words:
            if len(word) > 30:
                split_words = wordninja.split(word)
                fixed_words.extend(split_words)
            else:
                fixed_words.append(word)

        temp_data.append(fixed_words)

    return temp_data



# text_data02 = splited_data(text_data_corrected)
text_data02 = splited_data(text_data01)

text_data02[0]

['A',
 'few',
 'weeks',
 'ago',
 'I',
 'finally',
 'caved',
 'and',
 'called',
 'my',
 'doctor',
 'to',
 'get',
 'back',
 'on',
 'my',
 'anxiety',
 'medicine',
 'after',
 'my',
 'anxiety',
 'got',
 'so',
 'bad',
 'that',
 'I',
 'was',
 'having',
 'panic',
 'attacks',
 'whenever',
 'I',
 'left',
 'the',
 'house',
 'I',
 'cried',
 'picking',
 'it',
 'up',
 'I',
 'was',
 'so',
 'ready',
 'to',
 'feel',
 'normal',
 'again',
 'Well',
 'last',
 'night',
 'I',
 'went',
 'to',
 'the',
 'store',
 'and',
 'felt',
 'an',
 'anxiety',
 'attack',
 'coming',
 'Instead',
 'of',
 'spiraling',
 'out',
 'of',
 'control',
 'like',
 'I',
 'have',
 'been',
 'I',
 'was',
 'easily',
 'able',
 'to',
 'calm',
 'down',
 'I',
 'cant',
 'share',
 'this',
 'with',
 'anyone',
 'else',
 'so',
 'I',
 'thought',
 'to',
 'post',
 'it',
 'here',
 'I',
 'am',
 'so',
 'happy',
 'that',
 'I',
 'was',
 'able',
 'to',
 'finish',
 'shopping',
 'and',
 'that',
 'I',
 'didnt',
 'end',
 'up',
 'with',
 'a',
 'panic',
 'attack',
 

In [34]:
def lowercase(data):
    data_lowercase = []

    for corpus in data:
        temp_list = []
        for word in corpus:
            temp = word.lower()
            temp_list.append(temp)

        data_lowercase.append(temp_list)

    return data_lowercase

text_data03 = lowercase(text_data02)


text_data03[0]

['a',
 'few',
 'weeks',
 'ago',
 'i',
 'finally',
 'caved',
 'and',
 'called',
 'my',
 'doctor',
 'to',
 'get',
 'back',
 'on',
 'my',
 'anxiety',
 'medicine',
 'after',
 'my',
 'anxiety',
 'got',
 'so',
 'bad',
 'that',
 'i',
 'was',
 'having',
 'panic',
 'attacks',
 'whenever',
 'i',
 'left',
 'the',
 'house',
 'i',
 'cried',
 'picking',
 'it',
 'up',
 'i',
 'was',
 'so',
 'ready',
 'to',
 'feel',
 'normal',
 'again',
 'well',
 'last',
 'night',
 'i',
 'went',
 'to',
 'the',
 'store',
 'and',
 'felt',
 'an',
 'anxiety',
 'attack',
 'coming',
 'instead',
 'of',
 'spiraling',
 'out',
 'of',
 'control',
 'like',
 'i',
 'have',
 'been',
 'i',
 'was',
 'easily',
 'able',
 'to',
 'calm',
 'down',
 'i',
 'cant',
 'share',
 'this',
 'with',
 'anyone',
 'else',
 'so',
 'i',
 'thought',
 'to',
 'post',
 'it',
 'here',
 'i',
 'am',
 'so',
 'happy',
 'that',
 'i',
 'was',
 'able',
 'to',
 'finish',
 'shopping',
 'and',
 'that',
 'i',
 'didnt',
 'end',
 'up',
 'with',
 'a',
 'panic',
 'attack',
 

In [35]:
def remove_stopwords(data):

    res_data = []
    for corpus in data:
        temp_list=[]
        for word in corpus:
            if word not in stopwords.words("english") :
                temp_list.append(word)

        res_data.append(temp_list)

    return res_data
text_data04 = remove_stopwords(text_data03)



text_data04[0]

['weeks',
 'ago',
 'finally',
 'caved',
 'called',
 'doctor',
 'get',
 'back',
 'anxiety',
 'medicine',
 'anxiety',
 'got',
 'bad',
 'panic',
 'attacks',
 'whenever',
 'left',
 'house',
 'cried',
 'picking',
 'ready',
 'feel',
 'normal',
 'well',
 'last',
 'night',
 'went',
 'store',
 'felt',
 'anxiety',
 'attack',
 'coming',
 'instead',
 'spiraling',
 'control',
 'like',
 'easily',
 'able',
 'calm',
 'cant',
 'share',
 'anyone',
 'else',
 'thought',
 'post',
 'happy',
 'able',
 'finish',
 'shopping',
 'didnt',
 'end',
 'panic',
 'attack',
 'dont',
 'know',
 'waited',
 'long',
 'go',
 'back',
 'medicine']

In [51]:
print(len(train_dataset), len(eval_dataset))


8000 2000


In [36]:
from tokenizers import ByteLevelBPETokenizer

def bpe_encode_corpus(data, vocab_size=30000, min_frequency=2,
                      special_tokens=None):
    """
    data: لیستی از لیست‌های توکن (مثل text_data04)
    خروجی: لیست توکن‌های BPE برای هر جمله
    """
    if special_tokens is None:
        special_tokens = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]

    # 1. آماده‌سازی رشته‌ها
    raw_texts = [" ".join(tokens) for tokens in data]

    # 2. ساخت و آموزش ByteLevelBPETokenizer
    tokenizer = ByteLevelBPETokenizer()
    tokenizer.train_from_iterator(
        raw_texts,
        vocab_size=vocab_size,
        min_frequency=min_frequency,
        special_tokens=special_tokens
    )

    # 3. کدگذاری به BPE
    bpe_encoded = []
    for text in raw_texts:
        enc = tokenizer.encode(text)
        bpe_encoded.append(enc.tokens)

    return bpe_encoded

# استفاده:
bpe_texts = bpe_encode_corpus(text_data04)

# توکن‌های BPE جمله‌ی اول:
print(bpe_texts[0])


['weeks', 'Ġago', 'Ġfinally', 'Ġcaved', 'Ġcalled', 'Ġdoctor', 'Ġget', 'Ġback', 'Ġanxiety', 'Ġmedicine', 'Ġanxiety', 'Ġgot', 'Ġbad', 'Ġpanic', 'Ġattacks', 'Ġwhenever', 'Ġleft', 'Ġhouse', 'Ġcried', 'Ġpicking', 'Ġready', 'Ġfeel', 'Ġnormal', 'Ġwell', 'Ġlast', 'Ġnight', 'Ġwent', 'Ġstore', 'Ġfelt', 'Ġanxiety', 'Ġattack', 'Ġcoming', 'Ġinstead', 'Ġspiraling', 'Ġcontrol', 'Ġlike', 'Ġeasily', 'Ġable', 'Ġcalm', 'Ġcant', 'Ġshare', 'Ġanyone', 'Ġelse', 'Ġthought', 'Ġpost', 'Ġhappy', 'Ġable', 'Ġfinish', 'Ġshopping', 'Ġdidnt', 'Ġend', 'Ġpanic', 'Ġattack', 'Ġdont', 'Ġknow', 'Ġwaited', 'Ġlong', 'Ġgo', 'Ġback', 'Ġmedicine']


In [40]:
from sklearn.preprocessing import LabelEncoder
from transformers import BertForSequenceClassification, BertTokenizer


# اگر label_data لیست لیبل‌هاست:
le = LabelEncoder()
le.fit(label_data)

num_labels = len(le.classes_)

model = BertForSequenceClassification.from_pretrained(
    "my_bert_model",
    num_labels=num_labels
)

Some weights of the model checkpoint at my_bert_model were not used when initializing BertForSequenceClassification: ['fit_denses.1.weight', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.6.weight', 'fit_denses.3.bias', 'fit_denses.0.weight', 'fit_denses.3.weight', 'fit_denses.0.bias', 'fit_denses.5.bias', 'fit_denses.2.weight', 'fit_denses.4.weight', 'fit_denses.4.bias', 'cls.seq_relationship.weight', 'fit_denses.5.weight', 'fit_denses.2.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'fit_denses.6.bias', 'fit_denses.1.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are in

In [42]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }


In [48]:
from transformers import BertTokenizerFast, TrainingArguments, Trainer, DataCollatorWithPadding
import torch

# ۱. بارگذاری توکنایزر
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# ۲. تقسیم داده‌ها به آموزش و ارزیابی
splits = dataset.train_test_split(
    test_size=0.2,
    shuffle=True,
    seed=42
)
train_dataset = splits["train"]
eval_dataset  = splits["test"]

# ۳. تغییر نام ستون 'label' به 'labels' تا با Trainer هماهنگ شود
train_dataset = train_dataset.rename_column("label", "labels")
eval_dataset  = eval_dataset.rename_column("label", "labels")

# ۴. توکنیزه کردن داده‌ها و اضافه کردن 'input_ids' و 'attention_mask'
def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_fn, batched=True)
eval_dataset  = eval_dataset.map(tokenize_fn, batched=True)

# ۵. حذف ستون 'text' از داده‌ها پس از توکنایز کردن
train_dataset = train_dataset.remove_columns(["text"])
eval_dataset  = eval_dataset.remove_columns(["text"])

# ۶. تبدیل داده‌ها به فرمت torch.Tensor برای Trainer
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ۷. اطمینان از قرارگیری مدل روی CPU
device = torch.device("cpu")
model.to(device)

# ۸. Data collator برای padding پویا
data_collator = DataCollatorWithPadding(tokenizer)

# ۹. تنظیمات آموزش با حذف GPU و fp16
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.0,
    logging_dir="./logs",
    no_cuda=True,   # اجبار به استفاده از CPU
    fp16=False,     # غیرفعال کردن mixed-precision
)

# ۱۰. ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ۱۱. شروع فاین‌تونینگ head
trainer.train()


Map: 100%|████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 3650.46 examples/s]
***** Running training *****
  Num examples = 8000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.651300,1.613843,0.195500,0.063940
2,1.613600,1.609746,0.194500,0.063341
3,1.610800,1.610295,0.201500,0.067586
4,1.610100,1.609637,0.195500,0.063940
5,1.609600,1.609649,0.195500,0.063940


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-1000\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to ./results\checkpoint-1500
Configuration saved in ./results\checkpoint-1500\config.json


TrainOutput(global_step=2500, training_loss=1.6190668212890624, metrics={'train_runtime': 13661.3339, 'train_samples_per_second': 2.928, 'train_steps_per_second': 0.183, 'total_flos': 1324744857600000.0, 'train_loss': 1.6190668212890624, 'epoch': 5.0})

In [49]:

# ۱۰. ارزیابی نهایی
eval_results = trainer.evaluate()
print("Eval results:", eval_results)

# ۱۱. (اختیاری) ذخیرهٔ مدل فاین‌تون‌شده
trainer.save_model("my_bert_model_head_finetuned")
tokenizer.save_pretrained("my_bert_model_head_finetuned")


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


Saving model checkpoint to my_bert_model_head_finetuned
Configuration saved in my_bert_model_head_finetuned\config.json


Eval results: {'eval_loss': 1.6096493005752563, 'eval_accuracy': 0.1955, 'eval_f1': 0.06394019238812212, 'eval_runtime': 202.7427, 'eval_samples_per_second': 9.865, 'eval_steps_per_second': 0.617, 'epoch': 5.0}


Model weights saved in my_bert_model_head_finetuned\pytorch_model.bin
tokenizer config file saved in my_bert_model_head_finetuned\tokenizer_config.json
Special tokens file saved in my_bert_model_head_finetuned\special_tokens_map.json
tokenizer config file saved in my_bert_model_head_finetuned\tokenizer_config.json
Special tokens file saved in my_bert_model_head_finetuned\special_tokens_map.json


('my_bert_model_head_finetuned\\tokenizer_config.json',
 'my_bert_model_head_finetuned\\special_tokens_map.json',
 'my_bert_model_head_finetuned\\vocab.txt',
 'my_bert_model_head_finetuned\\added_tokens.json',
 'my_bert_model_head_finetuned\\tokenizer.json')

In [50]:

# اینچا داره میگه و بررسی میگه اگه فریز نگنیم چقدر دقت میشه2 تا لایه اخر ر
# 1. Unfreeze دو لایهٔ آخر Transformer
for name, param in model.bert.named_parameters():
    # لایه‌های encoder.layer.10 و encoder.layer.11 را باز کنید
    if "encoder.layer.10" in name or "encoder.layer.11" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# 2. نرخ یادگیری کمتری انتخاب کنید
training_args.learning_rate = 5e-5

# 3. full model fine-tuning
# trainer را دوباره با همین تنظیمات اجرا کنید.

trainer.train()


***** Running training *****
  Num examples = 8000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.609600,1.609649,0.195500,0.063940
2,1.609500,1.609649,0.195500,0.063940
3,1.609400,1.609649,0.195500,0.063940
4,1.609500,1.609649,0.195500,0.063940
5,1.609300,1.609649,0.195500,0.063940


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-1000\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to ./results\checkpoint-1500
Configuration saved in ./results\checkpoint-1500\config.json


TrainOutput(global_step=2500, training_loss=1.6094640380859375, metrics={'train_runtime': 7121.1941, 'train_samples_per_second': 5.617, 'train_steps_per_second': 0.351, 'total_flos': 1324744857600000.0, 'train_loss': 1.6094640380859375, 'epoch': 5.0})

In [ ]:
# from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

# # ۶. Data collator برای padding پویا
# data_collator = DataCollatorWithPadding(tokenizer)

# # ۷. تعیین hyper-parameters
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",     # ارزیابی در پایان هر Epoch
#     save_strategy="epoch",           # ذخیره مدل در پایان هر Epoch
#     learning_rate=2e-3,              # lr بالاتر چون فقط head را آپدیت می‌کنیم
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=5,
#     weight_decay=0.0,                # معمولاً برای head decay=0 مناسب است
#     logging_dir="./logs",
#     fp16=True,                       # mixed-precision برای سرعت بالاتر روی GPU
# )

# # ۸. ساخت Trainer
# trainer = Trainer(
#     model=model,                     # مدل بارگذاری‌شده و encoder فریزشده
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics, # تابع محاسبه متریک‌ها
# )

# # ۹. شروع فاین‌تونینگ head
# trainer.train()  همه رو بزار روی سی \ی یو